# FIRST HOMEWORK: LINEAR AND DISCRETE MODELS (TOPICS 1 AND 2)

## Data Science and Engineering



**Full name:** Laura Escobar Crespo
    


**Pre-needed code:**

As I have worked with Google Colab, the following code is needed to upload the excel file where the data is stored and to import Pyomo, which is needed to solve the optimization problem.

In [ ]:
from google.colab import files
 
files.upload()  

In [ ]:
!pip install -q pyomo

In [ ]:
!apt-get install -y -qq glpk-utils

**Pre-homework code and Explanation of the Excel file:**

The Excel file used for this assignment contains 150 rows and 13 columns.It contains information about a health insurance company. Each row represents a different person, whereas the columns represent differnt constants for each one of the people. 

Explanation of the columns:

*    *Column 0-Age:* It shows the age of each person.
*    *Column 1-Sex:* It tells the gender of the person (string constant).
*    *Column 2-BMI:* Body Mass Index of each person.
*    *Column 3-Children:* Number of kids that the person has.
*    *Column 4-Smoker:* It tells if the person smoke or not (1=smoker, 0=not a smoker).
*    *Column 5-Charges:* It shows the charges of each person, the money that the insurance company has to pay. It includes the cost of operations (it also includes "fast operation", like tooth withdrawals), medical consultations, additional medicines and special inquiries (for example radiographies).
*    *Column 6-Salary:* It contains the salary that a person earns in a month.
*    *Column 7-Cost Insurance:* It tell how much money do the people pay to the insurance in a month.
*    *Column 8-Number of times hospitalized:* Besides how many times the person has been interned in the hospital, it includes the number of "fast operations". This is constant sumarize the number of hospitalizations from a whole year.
*    *Column 9-Person:* Just to identify each person.
*    *Column 10-Years:* It shows how many years the person has been in the insurance.
*    *Column 11-Female:* It tells if the person is a woman or not (1=female, 0=male).
*    *Column 12-Male:* It tells if the person is a man or not (1=male, 0=female).

In [ ]:
import pandas as pd
data = pd.read_excel('dataset.xlsx')  

In [ ]:
data

,age,sex,bmi,children,smoker,charges,salary(by month),cost insutrance(by month),Number of times hostpitalized,Person,Years,female,male
0,19,female,27.900,0,1,16884.92400,1998,46,8,Person 1,5,1,0
1,18,male,33.770,1,0,1725.55230,1500,30,10,Person 2,1,0,1
2,28,male,33.000,3,0,4449.46200,1160,53,8,Person 3,1,0,1
3,33,male,22.705,0,0,21984.47061,1537,41,4,Person 4,7,0,1
4,32,male,28.880,0,0,3866.85520,926,85,10,Person 5,7,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,29,female,38.830,3,0,5138.25670,1799,95,6,Person 146,7,1,0
146,46,male,30.495,3,1,40720.55105,1318,84,6,Person 147,6,0,1
147,51,female,37.730,1,0,9877.60770,1069,30,2,Person 148,1,1,0
148,53,female,37.430,1,0,10959.69470,1373,94,0,Person 149,1,1,0


In [ ]:
Age = data.iloc[0:151,0]
Gender = data.iloc[0:151,1]
BMI = data.iloc[0:151,2]
Children = data.iloc[0:151,3]
Smoker = data.iloc[0:151,4]
Charges = data.iloc[0:151,5]
Salary = data.iloc[0:151,6]
Insurance = data.iloc[0:151,7]
N_hospitalized = data.iloc[0:151,8]
Person = data.iloc[0:151,9]
Years = data.iloc[0:151,10]
Female = data.iloc[0:151,11]
Male = data.iloc[0:151,12]

###**LINEAR MODEL**


---



**Description of the problem:**


The insurance company wants to know how many years they can renew the contract of insurance with each one of their old clients. They have to be carefull because some requirements have to be fulfilled as well as the objective of the company is satisfied. It has to be pointed out that the contract of renovation has a maximum of three years.

The objective of the company is to minimize the charges, which is the money that people spend in their health care (operations and medical consultations) and the insurance company has to pay.

To prevent people with charges too elevated in the future, the company has decided that no more than 180 hospitalizations in total are allowed. However, to prove that they are plenty professional and attend the customers, a minimum of 80 hospitalizations in total is required.

To ensure that the majority of clients have a good physical condition, the company looks at their BMI (Body Mass Index). The mean of the chosen clients cannot be greater than the media of the actual clients plus ten and cannot be less than the media of the actual clients minus ten. This constraint makes that the majority of the renewed clients have a BMI close to the media.

A new law has stablished that insurance companies have to support families. This law stablishes that at least 10% of the clients in each year have to be families with children.

The company also wants to keep some of the clients that have been more years with them. At least they want to keep some clients that have been with them 3 years.

Although the objective is to minimize the charges for the company, they want also to take a look on what the customers pay for the insurance. They want to achieve a 65% of the mean of the cost of the insurance. This constraint makes that the majority of the renewed clients are close to the 65% of the mean of insurance cost.





---
**Formulation of the problem:**

The *decision variables* are:

$$x_{i}=number\hspace{0.2cm}of\hspace{0.2cm}years\hspace{0.2cm}for\hspace{0.2cm}person_{i},\hspace{0.2cm}where\hspace{0.2cm}i\in [1,150]$$



The *objeective funcion* is defined as:
$$min\sum_{i=1}^{n} x_{i}·c_{i}$$



The *constraints* are the following:

\\

*    The first constraint is related with the number of hospitalizations:

  $$80\leq \frac{\sum_{i=1}^{n} x_{i}·h_{i}}{150} \leq 180$$

*    The second constraint is related with the BMI:

  $$\frac{\sum_{i=1}^{n} bmi_{i}}{150}-10\leq \frac{\sum_{i=1}^{n} x_{i}·bmi_{i}}{150} \leq \frac{\sum_{i=1}^{n} bmi_{i}}{150}+10$$

*    The third constraint is related with the number of children:

  $$ \frac{\sum_{i=1}^{n} x_{i}·k_{i}}{150} \geq \frac{\sum_{i=1}^{n} k_{i}}{150}·0.1$$

*    The fourth constraint is related with the number of years that a person has been in the insurance:

  $$ \frac{\sum_{i=1}^{n} x_{i}·y_{i}}{150} \geq 3$$


*    The fifth constraint is related with the cost of insurance for each person:

  $$ \frac{\sum_{i=1}^{n} x_{i}·r_{i}}{150} \geq \frac{\sum_{i=1}^{n}r_{i}}{150}·0.65 $$

where;


$$c_{i}=charges\hspace{0.2cm}of\hspace{0.2cm}person_{i}$$

$$h_{i}=number\hspace{0.2cm}of\hspace{0.2cm}hospitalizaton\hspace{0.2cm}of\hspace{0.2cm}person_{i}$$

$$c_{i}=bmi\hspace{0.2cm}of\hspace{0.2cm}person_{i}$$

$$k_{i}=number\hspace{0.2cm}of\hspace{0.2cm}children\hspace{0.2cm}of\hspace{0.2cm}person_{i}$$

$$y_{i}=number\hspace{0.2cm}of\hspace{0.2cm}years\hspace{0.2cm}in\hspace{0.2cm}the\hspace{0.2cm}insurance\hspace{0.2cm}of\hspace{0.2cm}person_{i}$$

$$r_{i}=cost\hspace{0.2cm}of\hspace{0.2cm}the\hspace{0.2cm}insurance\hspace{0.2cm}of\hspace{0.2cm}person_{i}$$

\\


*Note:* The number of hospitalizations is based on the past hospitalizations of each person, so the insurance company can get a idea of the health of the person and how much does it person cost. They can be lucky and the people chosed (that satisfy the constraints) keep the their record or even decrease it, or have bad luck and that suddently all the people increases their number of hospitalizations. This also happens with the  constat Charges, it is used to "predict" the behaviour of each person.



---

**Explanation of the code:**

After loading the Pyomo Modeling Enviroment, the *model* is defined. In this case the model is *Concrete* as it is inmediatily constructed and the used data	is	presented	at	the	time as the	components are	defined. It should be pointed out that the model is constructed using a local variable so it is easier to call when it is needed.

To create as many variables as people has the dataset, *RangeSet* defines the idex sets, so then we can define the decision varibles. The decison varibles, *x*, uses the index set previusly created with domanin *NonNegativeReals*, which means that in the solution we do not accept negative numbers. As in the description of the problem it was said that a single person cannot renew the contract for more than three years, we stablish a limit when the decision variables are created (stablishing bounds, where x can be any number that goes from 0 to 3).

The objective of is defined as a funcition, which is input is the model and just returns the sum of all the multiplication of the Charges of each person times their decion variable. Outside the ojective function, we apply it to the model, specifying that the objective is to minimize that value.

The next step is to define the constraints with Python. In *Formulation of the problem*, we have seen how these constraints are written mathematically. So it is basically transcript them into code. Each function represents in the code represents a constraint. Each function takes as input the model. Then, with a loop iterates through all the values of the constant doing the required calculations and chequing that the output of those calculation do not exceed or do not reach (greater or less than, respectively) the limit defined, returning the result. After that (and out of the function), we apply the constraint to our model. This procedure is repeted for all the constraints.

The constraints for the number of hospitalizations and the BMI are "*special*" as, if you look in the *Formulation of the problem* cell, they have an upper and a lower limint. To traduce this into code, we just have to separate each one of the constraints in two constrainst. One of the constraints will be limited with the maximum higher value (*hospital_constraint* and *bmi_constraint*), whereas the second will be limited by the minimum allowed value (*hospital_constraint_min* and *bmi_constraint_min*).  

In [ ]:
from pyomo.environ import *

# Create an instance of the model (using a local variable)
model = ConcreteModel()

#Define the index sets
model.person = RangeSet(1,len(Person)) 

#Define the decision variable
model.x = Var(model.person, domain=NonNegativeReals, bounds = (0,3))

#Define the objective function
def objective_func(model):
    return sum(Charges.values[i-1]*model.x[i] for i in model.person)

model.objective = Objective(rule=objective_func, sense=minimize)

#Define the constraints
def hospital_constraint(model):
    return sum(N_hospitalized.values[i-1]*model.x[i] for i in model.person)<=180 

model.hospital_constraint = Constraint(rule=hospital_constraint)

def hospital_constraint_min(model):
    return sum(N_hospitalized.values[i-1]*model.x[i] for i in model.person)>=80 

model.hospital_min = Constraint(rule=hospital_constraint_min)
def bmi_constraint(model):
    return (sum(BMI.values[i-1]*model.x[i] for i in model.person)/150)<=((sum(BMI.values[i-1] for i in model.person)/150)+10)

model.bmi = Constraint(rule=bmi_constraint)

def bmi_constraint_min(model):
    return (sum(BMI.values[i-1]*model.x[i] for i in model.person)/150)>=((sum(BMI.values[i-1] for i in model.person)/150)-10)

model.bmi_min = Constraint(rule=bmi_constraint_min)

def children_constraint(model):
    return (sum(Children.values[i-1]*model.x[i] for i in model.person))>=(sum(Children.values[i-1] for i in model.person)*0.1)

model.children = Constraint(rule=children_constraint)

def years_constraint(model):
    return (sum(Years.values[i-1]*model.x[i] for i in model.person)/150)>=3

model.years = Constraint(rule=years_constraint)

def mean_insurance_constraint(model):
    return (sum(Insurance.values[i-1]*model.x[i] for i in model.person)/150)>=((sum(Insurance.values[i-1] for i in model.person)/150)*0.65)

model.mean_insurance = Constraint(rule=mean_insurance_constraint)

In [ ]:
#Define the solver
opt = SolverFactory('glpk')

model.dual = Suffix(direction=Suffix.IMPORT)#for the sensitivity part

results = opt.solve(model)

model.display()

model.objective()


Model unknown

  Variables:
    x : Size=150, Index=person
        Key : Lower : Value            : Upper : Fixed : Stale : Domain
          1 :     0 :              0.0 :     3 : False : False : NonNegativeReals
          2 :     0 :              0.0 :     3 : False : False : NonNegativeReals
          3 :     0 :              0.0 :     3 : False : False : NonNegativeReals
          4 :     0 :              0.0 :     3 : False : False : NonNegativeReals
          5 :     0 :              0.0 :     3 : False : False : NonNegativeReals
          6 :     0 :              0.0 :     3 : False : False : NonNegativeReals
          7 :     0 :              3.0 :     3 : False : False : NonNegativeReals
          8 :     0 :              3.0 :     3 : False : False : NonNegativeReals
          9 :     0 : 2.63004219228584 :     3 : False : False : NonNegativeReals
         10 :     0 :              0.0 :     3 : False : False : NonNegativeReals
         11 :     0 :              0.0 :     3 : 

801688.3833670049



---

**Explanation of the results:**

If we look at the results of the decision varibles we see that we have a lot of ceros for a lot of people, which means that those people do not fullfil the constrainsts. On the other hand a lot of people will renew the contract for more three years, whereas others will have a mix of years and months. For example person 9, will renew the contract for 2.63 years which is equal to 2 years and 7 months. Person 91 will renew the contract for 2.34 years which is 2 years and 4 months. Person 139 renews for 2.74 years which is the same as 2 years and 8 months, while Person 148 firm a contract for 2.21 years which is equal to 2 years and 2 months. 

The total value of the charges for the company will be 801688.3833670049 euros/year.

It is interesting to take a look at the results of the constraints. For the two constraints of the number of hospitalizations, we see that the model takes as body value, a number closer to the upper limit. The contrary happens with the BMI constraints, it takes a value closer to the lower limit. For the yaers and insurance constraints it takes also body values close to the limits, whereas for the children constraint does not. This results about the constraints can give us hints about the next part of the assignment, the sensitivity of each constraint.

###**SENSITIVITIES OF THE CONSTRAINTS**


---

Analyse the sensitivity of a constraint help us to determine the importance of each one of the constraint and how they affect individually the result value of the objective function.

This anlalysis can be done manually, changing the limit value of each one of the constraints and running the model for each change, cheking the difference between the new result of the objective function and the old one. However, there exist a faster way of calculating it, and it is calculating the landa of each constraint when we are solving the model.

In the cell below, it is shown the landa of each one of the constraints. A positive landa means that if the constraint is aumented makes that the result of the objective function increases, while if the landa is negative it decreases. If it is cero, it means that the constraint is useless (it could be removed an basically will not affect to the final result).

As we see all the constraints seen to be very sensitive because they have big values except from the constraints for the upper limit of the BMI, the constraint for the lower limit constraint for the number of hospitalizations and the children constraint. This means that **for this dataset** are useless and can be removed. However if we have the same the same problem but with other dataset that, for example, very few people have children, the constraint with the children will be vary sensitive.

So, for that the sensitive of the constraints depends on the description of the problem, but also the depends on the dataset.


In [ ]:
model.dual.display()

dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key                 : Value
                    bmi :               0.0
                bmi_min :  56330.5829308732
               children :               0.0
    hospital_constraint : -4629.86165830521
           hospital_min :               0.0
         mean_insurance :  23579.0564393247
                  years :  37875.0653299728


###**DISCRETE MODEL**


---



**Description of the problem:**

For this problem, the insurance company has recived 150 solicitudes of 150 people that want to hire their services. The company has to check each profile and see if they are good for the business of the insurance. 

The company wants to maximize the goods of each months, which translates in how much money are they going to earn by the pays of the clients. However the company has to fullfil some requirements.

The company does not want to incorporate too much smokers, because in the futere that can implies a lot of health probles, so a lot of cost. For that they are not going to accept more that the 40% of the smokers of the new solicitudes.

As well, they want to be more or less that, at least the majority of their clients can afford the insurance, which means that the majority of the accepted clients have a salary equal or greater than 1200.

The company does not want a lot of clients to be too old and too young (too old, people greater than 55 years, and too young people with less than 26 years). They want that the majority of their clients are has closer has the media age. 

A law has stablish that for new acceptance of clients, at least a 45% of the new clients have to be women and another 45% have to be men, leaving the other 10% of free choice.







---
**Formulation of the problem:**

The *decision variables* are:

$$x_{i}=\left\lbrace\begin{array}{c}
1 & \textrm{accepted in the insurance}\\
0 & \textrm{otherwise}
\end{array}\right.$$

The *objeective funcion* is defined as:
$$max\sum_{i=1}^{n} x_{i}·w_{i}$$



The *constraints* are the following:


*    The first constraint is related with the number of smokers:

  $$ \sum_{i=1}^{n} x_{i}·m_{i} \leq \sum_{i=1}^{n} m_{i}*0.4$$

*    The second constraint is related with the salary of each person:

  $$\frac{\sum_{i=1}^{n} x_{i}·s_{i}}{150} \geq 1200$$

*    The third constraint is related with the age:

  $$ 26 \leq \frac{\sum_{i=1}^{n} x_{i}·a_{i}}{150} \leq 55$$

*    The fourth constraint is related with the number of women:

  $$ \sum_{i=1}^{n} x_{i}·f_{i}\geq \sum_{i=1}^{n} f_{i}·0.45$$

*    The third constraint is related with the number of men:

  $$ \sum_{i=1}^{n} x_{i}·v_{i}\geq \sum_{i=1}^{n} v_{i}·0.45$$

 
where;

$$m_{i}=if\hspace{0.2cm}person_{i}\hspace{0.2cm}is\hspace{0.2cm}a\hspace{0.2cm}smoker$$

$$s_{i}=salary\hspace{0.2cm}of\hspace{0.2cm}person_{i}$$

$$a_{i}=age\hspace{0.2cm}of\hspace{0.2cm}person_{i}$$

$$f_{i}=if\hspace{0.2cm}person_{i}\hspace{0.2cm}is\hspace{0.2cm}a\hspace{0.2cm}woman$$

$$v_{i}=if\hspace{0.2cm}person_{i}\hspace{0.2cm}is\hspace{0.2cm}a\hspace{0.2cm}man$$





---

**Explanation of the code:**

The code follows the same structure that the other from the *Linear Model*. It only changes the definition of the objective and constraint functions as a differnt problem is defined. Also, it changes the domain of the decision variables, as in this case it is a *Binary* problem, where the decision values can only be 1's or 0's. For the rest, it is practically the same as the previous exercise.

In [ ]:
# Create an instance of the model
model_binary = ConcreteModel()

#Define the index sets
model_binary.person = RangeSet(1,len(Person)) 

#Define the decision variable
model_binary.x = Var(model_binary.person, domain=Binary)

#Define the objective function
def objective_func_binary(model_binary):
    return sum(Insurance.values[i-1]*model_binary.x[i] for i in model_binary.person)

model_binary.objective = Objective(rule=objective_func_binary, sense=maximize)

#Define the constraints
def smoker_constraint(model_binary):
    return (sum(Smoker.values[i-1]*model_binary.x[i] for i in model_binary.person))<=(sum(Smoker.values[i-1] for i in model_binary.person)*0.4)

model_binary.smoker = Constraint(rule=smoker_constraint)

def salary_constraint(model_binary):
    return (sum(Salary.values[i-1]*model_binary.x[i] for i in model_binary.person)/150)>=1200

model_binary.salary = Constraint(rule=salary_constraint)

def age_constraint_min(model_binary):
    return (sum(Age.values[i-1]*model_binary.x[i] for i in model_binary.person)/150)>=26

model_binary.age_min = Constraint(rule=age_constraint_min)

def age_constraint_max(model_binary):
    return (sum(Age.values[i-1]*model_binary.x[i] for i in model_binary.person)/150)<=55

model_binary.age_max = Constraint(rule=age_constraint_max)

def female_constraint(model_binary):
    return (sum(Female.values[i-1]*model_binary.x[i] for i in model_binary.person))>=(sum(Female.values[i-1] for i in model_binary.person)*0.45)

model_binary.female = Constraint(rule=female_constraint)

def male_constraint(model_binary):
    return (sum(Male.values[i-1]*model_binary.x[i] for i in model_binary.person))>=(sum(Male.values[i-1] for i in model_binary.person)*0.45)

model_binary.male = Constraint(rule=male_constraint)

In [ ]:
#Define the solver
sol = SolverFactory('glpk')

results_binary = sol.solve(model_binary)

model_binary.display()

model_binary.objective()

Model unknown

  Variables:
    x : Size=150, Index=person
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   1.0 :     1 : False : False : Binary
          3 :     0 :   1.0 :     1 : False : False : Binary
          4 :     0 :   1.0 :     1 : False : False : Binary
          5 :     0 :   1.0 :     1 : False : False : Binary
          6 :     0 :   1.0 :     1 : False : False : Binary
          7 :     0 :   1.0 :     1 : False : False : Binary
          8 :     0 :   1.0 :     1 : False : False : Binary
          9 :     0 :   1.0 :     1 : False : False : Binary
         10 :     0 :   1.0 :     1 : False : False : Binary
         11 :     0 :   1.0 :     1 : False : False : Binary
         12 :     0 :   0.0 :     1 : False : False : Binary
         13 :     0 :   1.0 :     1 : False : False : Binary
         14 :     0 :   1.0 :     1 : False : False : Binary
         15 :     0 :   0.

8532.0



---

**Explanation of the results:**

If we look at the results of the decision varibles we see that we have a lot of ones compared with the number of zeros. This makes sense due to in our case qe are maximizing the objective function so, if we take as much people as we take the objective funtion result will be greater.  

The total value of the earnings (by month) for the company will be 8532.0 euros.

It is interesting to take a look at the results of the constraints. For the two constraints of the age, we see that the it takes a body value closer to the lower limit. The constraint for the number of smokers has to be very restrictive beacuse the body value is really close to the limit, while the one for the salary is a bit more flexible (but it still close to the limit). For the female and male constraint we see that the body value is far from the limits.